In [1]:
import os
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.parse import quote
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from operator import eq

In [2]:
def getBs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs = BeautifulSoup(html, 'html.parser')
    except AttributeError as e:
        return None
    return bs



In [15]:
## 크롤링 해야될 목록들
mainUrl = 'http://modernsweet.co.kr'

def get_category(url):
    bs = getBs(url)
    category = bs.ul.findAll('li',{'class':'xans-record-'})
    

    print('category list ')
    count = 1
    for item in category:
        print(count,')',item.a.get_text())
        count +=1
        
    while True:
        print('choice category (exit : q)')
        choice = input()
        for item in category:
            if eq(item.a.get_text(),choice):
                return item.a.attrs['href']
            if choice is 'q':
                return None
        print("can't find ",choice)
    

get_category(mainUrl)

category list 
1 ) BEST 50
2 ) ALL
3 ) OUTER
4 ) TOP
5 ) SHIRT
6 ) PANTS
7 ) BAG/SHOES
8 ) ACC
9 ) PERSONAL ORDER
choice category (exit : q)
q


In [4]:
## 다음 페이지 알아내기

def get_nextpage(url):
    bs = getBs(url)
    next_page = bs.find('img',{'alt':'다음 페이지'})
    next_page = next_page.parent.attrs['href']
    
    if eq(next_page,'#none'):
        return next_page
    
    andp = next_page.find('&') # 구분자 찾기
    next_page= next_page[andp:]
    return next_page

## 범위 벗어나면 #none
get_nextpage('http://modernsweet.co.kr/product/list.html?cate_no=70&page=5')

'&page=6'

In [5]:
## 페이지 내부에서 옷 목록 가져오기

def get_item(url):
    bs = getBs(url)
    item_list = bs.findAll('div',{'class':'box'})
    count = 1
    for item in item_list:
        try:
            print(count,') name : ',item.img.attrs['alt'])
            print('url :',item.a.attrs['href'])
            count+=1
        except AttributeError as e:
            print('no item')
    return item_list
item_list =  get_item('http://modernsweet.co.kr/product/list.html?cate_no=70')

1 ) name :  감탄 오버 반팔 티셔츠 20color
url : /product/detail.html?product_no=1187&cate_no=70&display_group=1
2 ) name :  마운틴 자수 하프 팬츠 4color
url : /product/detail.html?product_no=1186&cate_no=70&display_group=1
3 ) name :  230 슬림 일자 데님 팬츠(S~XL)
url : /product/detail.html?product_no=1185&cate_no=70&display_group=1
4 ) name :  아리조나 반팔 티 4color
url : /product/detail.html?product_no=1184&cate_no=70&display_group=1
5 ) name :  아레스 박스 오버 반팔티 3color
url : /product/detail.html?product_no=1183&cate_no=70&display_group=1
6 ) name :  SAROLEA 박스 나시티 3color
url : /product/detail.html?product_no=1182&cate_no=70&display_group=1
7 ) name :  타임 절개 카라 반팔티 8color
url : /product/detail.html?product_no=1181&cate_no=70&display_group=1
8 ) name :  코튼 워싱 카고 반바지 4color
url : /product/detail.html?product_no=1180&cate_no=70&display_group=1
9 ) name :  샌더스 반팔 니트 5color
url : /product/detail.html?product_no=1179&cate_no=70&display_group=1
10 ) name :  바이오워싱 코튼 반바지 3color
url : /product/detail.html?product_no=1178&cate_n

In [6]:
### 현제 페이지에서 이미지 모두 가져오기
downloadDirectory = 'download'

def getDownloadLink(source):
    mainUrl = 'http://modernsweet.co.kr'
    source = quote(source,safe=':/')
    return mainUrl+source

def get_img(url,filename):
    bs = getBs(url)
    downpath = 'downdown/'+filename+'/'
    item_information = bs.find('div',{'id':'prdDetail'})
    img_list = item_information.findAll('img')
    count = 1
    for img in img_list:
        print(getDownloadLink(img.attrs['ec-data-src']))
        urlretrieve(getDownloadLink(img.attrs['ec-data-src']),downpath+str(count)+'.jpg')
        count+=1


In [7]:
def get_information(url):
    bs = getBs(url)
    info_list = bs.find('div',{'class':'cont'})
    info_list = info_list.findAll('span')
    count = 1
    text = ''
    for info in info_list:
        #print(count,':',info.get_text())
        #count+=1
        text += info.get_text()+'\n'
    return text

print(get_information('http://modernsweet.co.kr/product/detail.html?product_no=1180&cate_no=70&display_group=1'))


INFO
INFO
개인적으로도 강력 추천드리는 반바지 입니다:)
카고 형식으로 제작된 반바지에요.
탄탄한 면 소재이지만 답답하지 않고 짧은 기장감으로 제작되어
시원하고 캐쥬얼한 느낌으로 착용하기 좋으며
기본 무지 티셔츠와 매치하여도 
카고 디테일(포켓) 이 있어서 포인트를 주기 좋은 제품입니다:)
무엇보다 봄,가을 시즌 되면
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!

무더운 여름에는 반팔티와 슬라브티 등을 매치하여
가볍고 편안하게 입으시고
FW 시즌되면 맨투맨과 티셔츠를 레이어드하여
남친룩을 연출하셔도 좋습니다:)
원단 자체가 워싱이 되어 제작된 제품이기 때문에
세탁 후에도 변형이나 뒤틀림이 전혀 없는 제품이라 매 시즌마다 꺼내어 입어주셔도 좋으며
허리 부분엔 뒷 밴딩으로 처리되어 있어 착용감도 굉장히 편하고
사이즈 또한 S,M,L,XL 까지 있어 남성분들 사이즈 선택에 폭도 넓습니다!
개인적으로는 컬러는 그레이, 블랙 제품이 어떤 상의와도 매치하기가 쉬워 추천드리며
남성분들 하나쯤 꼭 소장하시길 추천드리는 꿀템이라 생각합니다!
맨투맨과 연출된 코디컷은 곧 추가해서 보여드릴게요:)
COLOR
블랙, 오렌지, 화이트, 그레이
 
SIZE(cm)
S : 34(뒷밴딩) / 밑위 26 / 허벅지 32 / 밑단 28 / 총기장 39
M : 36(뒷밴딩) / 밑위 28 / 허벅지 33 / 밑단 29 / 총기장 40
L : 38(뒷밴딩) / 밑위 30 / 허벅지 34 / 밑단 30 / 총기장 41
XL : 40(뒷밴딩) / 밑위 32 / 허벅지 35 / 밑단 31 / 총기장 42
측정 위치에 따라 1-3CM 의 오차가 발생 할 수 있습니다.
FABRIC
면/스판
MODEL
174 CM / 63 KG
평소 28~29사이즈 착용 모델
S 사이즈 편안하게 착용하였습니다:)
DETAIL
비침 / 없음
두께감 / 보통
신축성 / 약간
그레이 / S 사이즈
DETAIL VIEW
화이트 / 그레이 / 오렌지 / 

In [8]:
def get_infofile(text,filename):

    path = 'downdown/'+filename+'/'
    f = open(path+filename+'.txt','w',-1, "utf-8")
    
    f.write(text)
    f.close()
    

In [17]:
def crawling():
    global mainUrl
    ## 메뉴 가져오기
    category = get_category(mainUrl)
    if category is None:
        return None
    ##메뉴 크롤링 시작
    print('start crawling')
    target = mainUrl+category
    count = 1
    while 1:
        print(count,' page')
        ##타겟 페이지 내부의 상품 추출
        item_list = get_item(target)
        for item in item_list:
            try:
                itemName = item.img.attrs['alt']
                itemUrl = item.a.attrs['href']
            except AttributeError as e:
                print('no item')
                break
                    
            os.makedirs('downdown/'+itemName)
            get_img(mainUrl+itemUrl,itemName)
            text = get_information(mainUrl+itemUrl)
            get_infofile(text,itemName)
            
            
        if eq(get_nextpage(target),'#none'):
            break
        else:
            target = target+get_nextpage(target)
        count += 1

In [19]:
crawling()

category list 
1 ) BEST 50
2 ) ALL
3 ) OUTER
4 ) TOP
5 ) SHIRT
6 ) PANTS
7 ) BAG/SHOES
8 ) ACC
9 ) PERSONAL ORDER
choice category (exit : q)
8
can't find  8
choice category (exit : q)
ACC
start crawling
1  page
1 ) name :  하프클리어 뿔테 안경
url : /product/detail.html?product_no=1165&cate_no=43&display_group=1
2 ) name :  소가죽 롱 벨트 5color
url : /product/detail.html?product_no=1158&cate_no=43&display_group=1
3 ) name :  센서블 투명 뿔테 안경
url : /product/detail.html?product_no=1156&cate_no=43&display_group=1
4 ) name :  모던스윗 선글라스 4color
url : /product/detail.html?product_no=1149&cate_no=43&display_group=1
5 ) name :  꽈배기 캐주얼 벨트 4color
url : /product/detail.html?product_no=1140&cate_no=43&display_group=1
6 ) name :  라운드 버클 벨트 4color
url : /product/detail.html?product_no=1100&cate_no=43&display_group=1
7 ) name :  캐주얼 소가죽 각 버클 벨트
url : /product/detail.html?product_no=1098&cate_no=43&display_group=1
8 ) name :  B 자수 로고 볼캡
url : /product/detail.html?product_no=1064&cate_no=43&display_group=1
9 ) name :  C

KeyboardInterrupt: 